<a href="https://colab.research.google.com/github/limkaram/SKtelecom-TANGO-keyword-analysis/blob/master/TANGO_%EA%B5%AC%EC%B6%95_%EC%9E%91%EC%97%85_%ED%82%A4%EC%9B%8C%EB%93%9C_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Drive 목록 Mounting(매 코딩 시작전 해주어야함)
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_excel('/content/drive/My Drive/Personal Project/TANGO 구축 작업 키워드 분석/20200101-20200731_TANGO 회선영향없음 키워드 주간 구축 작업.xlsx')
# 분석 예정 컬럼 = ['작업명']
data['작업명'].loc[4393]  # 만들어 놓은 정규식 패턴에 맞지 않는 작업명

In [ ]:
# 정규식 패턴에 부합하는 작업명 추출 함수
import re
def get_regex_matching_text(text):
    text_in_square_backet_regex = re.compile(r'\[{1}.+\]{1}')  # 정규식 패턴 : 대괄호내 어떠한 단어라도 들어가있으면 부합
    matched_text = text_in_square_backet_regex.search(text)
    if matched_text:  # 정규식 패턴에 부합하는 작업명을 지닌 경우
        return matched_text.group()
    else:  # 정규식 패턴에 부합하는 부분을 전혀 가지지 않은 작업명의 경우 None을 반환하므로, pass
        pass

In [ ]:
data[data['작업명'].apply(get_regex_matching_text).isnull() == True]  # 정규식패턴에 맞지 않는 row 확인(index = 4393)
data['정제_작업명'] = data['작업명'].apply(get_regex_matching_text)  # 정규식화 함수를 거친 새로운 columns 생성

# 대괄호 제거 후 내부 단어만 추출
def get_rid_of_square_bracket(text):
    if text:  # refined_text가 None이 아닌 경우만 대괄호 제거 후 정제
        return text.replace('][', '|').lstrip('[').rstrip(']').split('|')
    else:
        return 0
data['정제_작업명'] = data['정제_작업명'].apply(get_rid_of_square_bracket)
data['정제_작업명'].loc[4390:4400]

In [ ]:
# '정제_작업명' column에서 회선영향없음 포함하는 값은 삭제
def re_refinement(value):
    if type(value) is list:
        return [i for i in value if len(i) != 1 if '회선영향없음' not in i]
    else:
        pass

data['정제_작업명'] = data['정제_작업명'].apply(re_refinement)  # 최종 정제된 단어 리스트를 value로 가지는 column 완성